In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

SEED = 515
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Use Pre-Trained Embeddings

In [2]:
import torchtext
from torchtext.data import Field, LabelField, BucketIterator

# Set `include_lengths=True` to return the text-lengths too. 
TEXT = Field(tokenize='spacy', tokenizer_language='en_core_web_sm', include_lengths=True)
LABEL = LabelField(dtype=torch.float)

train_data, test_data = torchtext.datasets.IMDB.splits(TEXT, LABEL, root="../assets/data")
train_data, valid_data = train_data.split()

In [3]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE, 
                 vectors="glove.6B.100d", vectors_cache="../assets/vector_cache", 
                 unk_init=torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [4]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# NOTE: `sort_within_batch=True` for packed padded sequences
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE, sort_within_batch=True, device=device)

# Build Model
## Different RNN Architecture: LSTM
## Bidirectional RNN
## Multi-Layer RNN
## Regularization: Dropout

## Implementation Details
### How to ignore the `<pad>` token?
* Pass the index of `<pad>` token as `padding_idx` argument to `nn.Embedding`
* Pass packed padded sequences to RNNs
* Attention?
* `ignore_index` of loss function?

### The `hidden` output of LSTM
The `hidden` has a shape of **(num_layers * num_directions, batch_size, hid_dim)**.  
These are ordered in **(forward_layer_0, backward_layer_0, forward_layer_1, backward_layer_1, ..., forward_layer_n, backward_layer_n)**, from bottom to top.  

In [5]:
class Classifier(nn.Module):
    def __init__(self, in_dim, emb_dim, hid_dim, out_dim, n_layers, bidirect, dropout, pad_idx):
        super().__init__()
        self.emb = nn.Embedding(in_dim, emb_dim, padding_idx=pad_idx)
        self.rnn = nn.LSTM(emb_dim, hid_dim, num_layers=n_layers, 
                           bidirectional=bidirect, dropout=dropout)
        # `bidirectional=True` by default
        self.fc = nn.Linear(hid_dim*2, out_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lens):
        # text: (step, batch)
        embedded = self.dropout(self.emb(text))
        # Pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lens)
        # hidden: (num_layers*num_directions, batch, hid_dim)
        packed_outs, (hidden, cell) = self.rnn(packed_embedded)
        # Unpack sequence, NOT used here. 
        outs, out_lens = nn.utils.rnn.pad_packed_sequence(packed_outs)

        hidden = self.dropout(torch.cat([hidden[-2], hidden[-1]], dim=-1))
        return self.fc(hidden)

In [6]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

IN_DIM = len(TEXT.vocab)
EMB_DIM = 100
HID_DIM = 256
OUT_DIM = 1
N_LAYERS = 2
BIDIRECT = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

classifier = Classifier(IN_DIM, EMB_DIM, HID_DIM, OUT_DIM, 
                        N_LAYERS, BIDIRECT, DROPOUT, PAD_IDX).to(device)
count_parameters(classifier)

4810857

### Initialize Embeddings with Pre-Trained Vectors
* Replace the embedding layer's weight with `TEXT.vocab.vectors`.  
    * **Note:** Use `weight.data` for data Initialization.  
* Explicitly Initialize the embeddings of `<unk>` and `<pad>` tokens to zeros.  
    * Because the `padding_idx` has been passed to `nn.Embedding`, so the `<pad>` embedding will remain zeros throughout training.  
    * While the `<unk>` embedding will be learned.

In [7]:
print(TEXT.vocab.vectors.size())
print(classifier.emb.weight.size())

print(TEXT.vocab.vectors[:5, :8])
print(classifier.emb.weight[:5, :8])
classifier.emb.weight.data.copy_(TEXT.vocab.vectors)
print(classifier.emb.weight[:5, :8])

torch.Size([25002, 100])
torch.Size([25002, 100])
tensor([[-0.3902,  1.4256,  0.0491, -0.0559, -1.0172,  0.6198,  0.3173, -1.8878],
        [-0.4843,  1.6366, -1.4119, -0.4014, -1.1278,  1.6279,  0.7653, -0.5195],
        [-0.0382, -0.2449,  0.7281, -0.3996,  0.0832,  0.0440, -0.3914,  0.3344],
        [-0.1077,  0.1105,  0.5981, -0.5436,  0.6740,  0.1066,  0.0389,  0.3548],
        [-0.3398,  0.2094,  0.4635, -0.6479, -0.3838,  0.0380,  0.1713,  0.1598]])
tensor([[ 1.0413, -0.3617, -1.4266, -0.1845,  2.0739, -1.1237,  0.1582,  0.0154],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 1.3434,  0.2367,  1.2310,  1.6074, -0.0450,  1.5775, -1.2775,  0.1296],
        [ 0.5770,  0.1806,  0.6710,  0.3620, -1.0804, -1.2183, -1.7412, -0.4920],
        [-0.9820, -0.1802, -1.6491, -0.3869, -0.4613,  0.6794, -1.5137, -0.2834]],
       device='cuda:0', grad_fn=<SliceBackward>)
tensor([[-0.3902,  1.4256,  0.0491, -0.0559, -1.0172,  0.6198,  0.3173, -1.8878

In [8]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

classifier.emb.weight.data[UNK_IDX].zero_()
classifier.emb.weight.data[PAD_IDX].zero_()

print(classifier.emb.weight[:5, :8])

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281, -0.3996,  0.0832,  0.0440, -0.3914,  0.3344],
        [-0.1077,  0.1105,  0.5981, -0.5436,  0.6740,  0.1066,  0.0389,  0.3548],
        [-0.3398,  0.2094,  0.4635, -0.6479, -0.3838,  0.0380,  0.1713,  0.1598]],
       device='cuda:0', grad_fn=<SliceBackward>)


# Train Model

In [9]:
optimizer = optim.Adam(classifier.parameters())
# Binary cross entropy with logits. 
# The binary version of cross entropy loss. 
loss_func = nn.BCEWithLogitsLoss().to(device)

In [10]:
def train_epoch(classifier, iterator, optimizer, loss_func):
    classifier.train()
    epoch_loss = 0
    epoch_acc = 0
    for batch in iterator:
        # Forward pass
        text, text_lens = batch.text
        preds = classifier(text, text_lens).squeeze(-1)
        # Calculate loss
        loss = loss_func(preds, batch.label)
        # Backward propagation
        optimizer.zero_grad()
        loss.backward()
        # Update weights
        optimizer.step()
        # Accumulate loss and acc
        epoch_loss += loss.item()
        epoch_acc += (torch.round(torch.sigmoid(preds)) == batch.label).sum().item() / preds.size(0)
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

def eval_epoch(classifier, iterator, loss_func):
    classifier.eval()
    epoch_loss = 0
    epoch_acc = 0
    with torch.no_grad():
        for batch in iterator:
            # Forward pass
            text, text_lens = batch.text
            preds = classifier(text, text_lens).squeeze(-1)
            # Calculate loss
            loss = loss_func(preds, batch.label)
            # Accumulate loss and acc
            epoch_loss += loss.item()
            epoch_acc += (torch.round(torch.sigmoid(preds)) == batch.label).sum().item() / preds.size(0)
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

In [11]:
import time
N_EPOCHS = 10
best_valid_loss = np.inf

for epoch in range(N_EPOCHS):
    t0 = time.time()
    train_loss, train_acc = train_epoch(classifier, train_iterator, optimizer, loss_func)
    valid_loss, valid_acc = eval_epoch(classifier, valid_iterator, loss_func)
    epoch_secs = time.time() - t0

    epoch_mins, epoch_secs = int(epoch_secs // 60), int(epoch_secs % 60)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(classifier.state_dict(), "models/tut2-model.pt")
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 56s
	Train Loss: 0.646 | Train Acc: 61.75%
	 Val. Loss: 0.626 |  Val. Acc: 69.46%


In [1]:
classifier.load_state_dict(torch.load("models/tut2-model.pt"))

valid_loss, valid_acc = eval_epoch(classifier, valid_iterator, loss_func)
test_loss, test_acc = eval_epoch(classifier, test_iterator, loss_func)

print(f'Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}%')
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

NameError: name 'classifier' is not defined

# User Input

In [2]:
import spacy
nlp = spacy.load('en')

def predict_senti(classifier, sentence):
    classifier.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[tok] for tok in tokenized]
    lens = len(indexed)

    indexed = torch.tensor(indexed, dtype=torch.long).unsqueeze(1).to(device)
    lens = torch.tensor(lens, dtype=torch.long).unsqueeze(0).to(device)
    pred = torch.sigmoid(classifier(indexed, lens)).round().type(torch.long)
    return LABEL.vocab.itos[pred.item()]

OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [3]:
predict_senti(classifier, "This is a good film.")

NameError: name 'predict_senti' is not defined

In [4]:
predict_senti(classifier, "This film is terrible.")

NameError: name 'predict_senti' is not defined

# Check Embeddings
* The Embeddings of `<unk>` and `<pad>` tokens
    * Because the `padding_idx` has been passed to `nn.Embedding`, so the `<pad>` embedding will remain zeros throughout training.  
    * While the `<unk>` embedding will be learned.

In [5]:
print(classifier.emb.weight[:5, :8])

NameError: name 'classifier' is not defined